In [98]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import IsolationForest
import joblib

time_freq = '15s'
image_length = 40

In [99]:
# Functions Cell

# Function to transform DataFrame from packets to packets per time_group variable.
# Returns DataFrame with MAC addresses, average volumes, and average delta times 
# of packets every time_group 
# Input: DataFrame, string (MAC address)
# Return DataFrame
def transform(df, mac):
    df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
    packets_per_min = df.set_index('Epoch Time').groupby(pd.Grouper(freq=time_freq))['Length'].mean()
    packets_per_min = packets_per_min.fillna(0)

    time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq=time_freq))['Delta Time'].mean()
    time_deltas = time_deltas.fillna(time_deltas.mean())
    df = pd.merge(packets_per_min, time_deltas, left_index=True, right_index=True)

    df['MAC Source'] = mac
    return df

# Function to train and save an Isolation Forest model for a specific MAC address.
# This model is used to return outliers for input time frequencies
# Input: DataFrame, string (MAC address)
# Return: void
def train_and_save_S_model(df, mac):
    #df = df[df['MAC Source']==mac]
    # df = transform(df)

    X = df[['Delta Time']].values
    x_str = [str(i) for i in X]
    clf = DecisionTreeClassifier().fit(np.array(X).reshape(-1,1), x_str)
    
    filename = '../models3'
    joblib.dump(clf, filename + '\\' + mac.replace(':','') + '.sav')


# Function that takes an IsolationForest model and an "image," or time frequency, of data
# and detects the outliers.
# Input: DataFrame, model (IsolationForest)
def detect_outliers(df, comparison_model):
    X = df['Delta Time'].values
    y_pred = model.predict(np.array(X).reshape(-1,1))
    # print(y_pred)
    y_pred = [float(i.strip('][')) for i in y_pred]

    return y_pred # percentage of outliers


# Function that takes two time interval images of data and an outlier model, returning the absolute
# difference in outliers between the two.
# Input: DataFrame (anchor image), DataFrame (test image), model (IsolationForest) 
def compare_models(df_1, df_2, outlier_model, label):

    x = detect_outliers(df_1, outlier_model)
    y = detect_outliers(df_2, outlier_model)
    print(np.sum(abs(np.array(x)-np.array(y))), label)

    return np.sum(abs(np.array(x)-np.array(y))) #((abs(percent_of_outliers_2 - percent_of_outliers_anchor))/percent_of_outliers_anchor)*100


# Function to extract the time groups (images) of data from a given dataset (network traffic).
# The input data should only be for one MAC, if not, it should be sorted where it can be grouped.
# e.g. if time_freq = '15s' and image_length = 60, the time frame is 15 minutes of data per image.
#
# Input: DataFrame
# Return: List of Dataframes 
def extract_images_and_labes(df):
    df = pd.DataFrame(df)
    df['Time'] = df.index
    df.index = pd.RangeIndex(len(df.index))
    groups = df.groupby([df.index // image_length])
    
    mac_images = []
    mac_labels = []
    for _, g in groups:

        image = g[['Delta Time']]
        rows_needed = image_length-len(image)
        for _ in range(rows_needed):
            image.loc[len(df.index)] = [image.mean()] 
        mac_images.append(image)
        mac_labels.append(g['MAC Source'].unique()[0])
    return mac_images, mac_labels

# Function to transform data into time intervals, then return grouped time frequencies
# and labels from the input DataFrame.
# Input: DataFrame
# Return: List of DataFrames, List of Strings
def transform_data(df):
    macs = ['00:0c:29:9d:9e:9e','00:80:f4:09:51:3b','48:5b:39:64:40:79','00:0c:29:e6:14:0d']
    df = df[df['MAC Source'].isin(macs)]
    grouped_mac = df.groupby(df['MAC Source'])
    mac_arr_df = [grouped_mac.get_group(d) for d in df['MAC Source'].unique()]

    images = []
    labels = []
    for df in mac_arr_df:
        df['Epoch Time'] = pd.to_datetime(df['Epoch Time'], errors='coerce')
        packets_per_min = df.set_index('Epoch Time').groupby(pd.Grouper(freq=time_freq))['No.'].count()
        packets_per_min = packets_per_min.fillna(0)

        time_deltas = df.set_index('Epoch Time').groupby(pd.Grouper(freq=time_freq))['Delta Time'].mean()
        time_deltas = time_deltas.fillna(time_deltas.mean())
        df_final = pd.merge(packets_per_min, time_deltas, left_index=True, right_index=True)

        # Assigns the MAC address to the 
        df_final['MAC Source'] = df['MAC Source'].unique()[0]

        df = df_final[:-1] # get rid of extra row at end
        imgs, lbls = extract_images_and_labes(df_final)
        images += imgs
        labels += lbls

    return images, labels


# Function to create positive and negavite combinations of images for the 
# Desicion Tree model training phase. It takes both images and labels from specific MAC
# and all MACs to create both the positive and negative image pairs. 
# The dimensions of the pairs is (len(images)*2, 2, image_length).
# e.g. (90, 2, 60): 90 pairs, 2 columns (features), 60 rows
#
# len(images)*2: for each images a positive and negative match are generated, hence the times 2
# 2: two columns for each feature - avg volume, avg time delta
# image length: how many rows in the image (depending on the time groupings and frequency)
# 
# Inupt: List of DataFrames, List of Strings, List of DataFrames, List of Strings
# Return: List of Pair of Lists, List of Strings
def make_pairs(images, labels, images_extra, labels_extra):
    
    pairImages = []
    pairLabels = []

    df = pd.DataFrame([i for i in range(len(images))], columns=['idx'])
    df['label'] = labels

    df_extra = pd.DataFrame([i for i in range(len(images_extra))], columns=['idx'])
    df_extra['label'] = labels_extra

    for idx, image in enumerate(images):
        label = labels[idx]
        
        pos_df = df
        pos_img_idx = pos_df.sample().to_numpy()[0][0]
        pos_img = images[pos_img_idx]
        pairImages.append((image, pos_img))
        pairLabels.append(['Normal'])

        neg_df = df_extra[df_extra['label'] != label]
        neg_img_idx = neg_df.sample().to_numpy()[0][0]
        neg_img = images_extra[neg_img_idx]
        pairImages.append((image, neg_img))
        pairLabels.append(['Rouge'])
    
    return (pairImages, pairLabels)

# Function to train and save the comparison model. This takes the encodings (or absolute differences) 
# of two devices and the labels mapped to the encodings (whether or not the images match). The model
# then trains and recognizes what device images match and which do not.
# Input: List of ints, List of strings, string (MAC address)
def train_and_save_C_model(encodings, labels, mac):
    model = DecisionTreeClassifier()
    model.fit(encodings, labels)

    filename = '../models3' + mac.replace(':','') + '_compare.sav'
    joblib.dump(model, filename)
    return model
    



In [100]:
# Train and save the outlier models for each auth device

data = '../../test_data/csv2/eth2dump-clean-6h_1.csv'
df = pd.read_csv(data)
df = df[(df['MAC Source']=='00:0c:29:e6:14:0d') | (df['MAC Source']=='00:0c:29:9d:9e:9e') | (df['MAC Source']=='48:5b:39:64:40:79') | (df['MAC Source']=='00:80:f4:09:51:3b')]

grouped_ip = df.groupby(df['MAC Source'])
macs_arr = [grouped_ip.get_group(d) for d in df['MAC Source'].unique()]
# iterate through each mac/device
for mac in macs_arr:
    name = mac['MAC Source'].unique()[0]
    df = transform(mac, name)
    # save model
    train_and_save_S_model(df, name)



In [101]:
# start to train the DT difference models for each auth device

# iterate through clean data and compbine into one DataFrame, also grouping the data by time frequency declared by time_freq
import glob
dfs = []
files = glob.glob('../../test_data/csv2/*.csv')
for file in files[0:3]:
    print(file)
    macs = ['00:0c:29:9d:9e:9e','00:80:f4:09:51:3b','48:5b:39:64:40:79','00:0c:29:e6:14:0d']
    df = pd.read_csv(file)
    df = df[df['MAC Source'].isin(macs)]
    grouped_ip = df.groupby(df['MAC Source'])
    macs_arr = [grouped_ip.get_group(d) for d in df['MAC Source'].unique()]
    macs = []
    for mac in macs_arr:
        label = mac['MAC Source'].unique()[0]
        mac = transform(mac, label)
        macs.append(mac[:-1])
    df = pd.concat(macs)
    dfs.append(df)
df = pd.concat(dfs)

C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-0,5h_1.csv
C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-1h_1.csv
C:\Users\carlo\Documents\College\reu_cyber\test_data\csv2\eth2dump-clean-6h_1.csv


In [102]:
# extract all time frame images for negative matches used later
all_images, all_labels = extract_images_and_labes(df)
len(all_images)

135

In [103]:
# group clean data by MAC address
grouped_ip = df.groupby(df['MAC Source'])
macs_arr = [grouped_ip.get_group(d) for d in df['MAC Source'].unique()]

# EXPLAINATION:

# iterate through MACs 
# for each mac: 
#   get its time interval images 
#   get both positive and negative matches for training, as well as the labels indicating their relationship
#   for each pair/match:
#       get absolute difference between the two
#   train the DT on the absolute difference encodins (similarity score) and their labels (match or not)

# models should now know which image differences are similar and declared positive, or vice verse 


for mac_df in macs_arr:
    mac_label = mac_df['MAC Source'].unique()[0]
    print('mac_df =',mac_label)
    mac_images, mac_labels = extract_images_and_labes(mac_df)
    
    pairs, labels = make_pairs(mac_images, mac_labels, all_images, all_labels)
    
    filename = '../models3'
    model = joblib.load(filename + '\\' + mac_label.replace(':','') + '.sav')

    encodings = []
    labels_return = []
    for pair, label in zip(pairs,labels):
        # print(np.array(pair[0]).shape, np.array(pair[1]).shape)
        result = [compare_models(pair[0], pair[1], model, label[0])]
        encodings.append(result)
        labels_return.append(label[0])
    model = train_and_save_C_model(encodings, labels_return, mac_label)
    

mac_df = 00:0c:29:9d:9e:9e
0.22870738000000013 Normal
2.1108565599999998 Rouge
0.22363688000000007 Normal
2.04174958 Rouge
0.17907486000000006 Normal
2.1365554399999995 Rouge
0.22177024999999997 Normal
1.8906531899999999 Rouge
0.17792994000000012 Normal
1.9683429899999998 Rouge
0.18545605999999998 Normal
2.0060359199999995 Rouge
0.15881212999999997 Normal
1.9861269499999996 Rouge
0.3052638700000001 Normal
1.9526881099999998 Rouge
0.16428498999999996 Normal
1.9956478599999998 Rouge
0.14157820999999998 Normal
2.05232479 Rouge
0.16999951 Normal
2.0122746299999994 Rouge
0.20952981999999998 Normal
2.1095672599999995 Rouge
0.16543991999999996 Normal
2.07395936 Rouge
0.19325257999999998 Normal
1.9177359699999998 Rouge
0.1705362700000001 Normal
2.06094492 Rouge
0.19901367000000003 Normal
2.0587464699999996 Rouge
0.22320378999999985 Normal
2.0106440599999997 Rouge
0.24248627 Normal
2.04584653 Rouge
0.23261166000000005 Normal
2.0720304799999996 Rouge
0.2991552200000001 Normal
1.9587631599999997 

ValueError: operands could not be broadcast together with shapes (40,) (38,) 